# Assignment 4 #

We are given three columns of integers with a row for each node. The first two columns contain x and y coordinates of the node positions in a plane. The third column contains node costs. The goal is to select exactly 50% of the nodes (if the number of nodes is odd we round the number of nodes to be selected up) and form a Hamiltonian cycle (closed path) through this set of nodes such that the sum of the total length of the path plus the total cost of the selected nodes is minimized. The distances between nodes are calculated as Euclidean distances rounded mathematically to integer values. The distance matrix should be calculated just after reading an instance and then only the distance matrix (no nodes coordinates) should be accessed by optimization methods to allow instances defined only by distance matrices. 

## Read the data ##

In [1]:
import pandas as pd
import numpy as np
from numpy.typing import ArrayLike, NDArray
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from math import sqrt
from typing import List, Optional, Tuple, Union 

In [2]:
# read data into dataframes
instances = {
    "A": pd.read_csv("data/TSPA.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "B": pd.read_csv("data/TSPB.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "C": pd.read_csv("data/TSPC.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "D": pd.read_csv("data/TSPD.csv", sep=';', header=None, names=["x", "y", "cost"]),
}

In [3]:
def calculate_distance_matrix(df: pd.DataFrame) -> NDArray[np.int32]:
    """
    Calculate the distance matrix from the dataframe.
    The dataframe contains 'x' and 'y' columns for the coordinates.
    The distances are Euclidean, rounded to the nearest integer + the cost of the destination node.
    """
    coordinates = df[['x', 'y']].to_numpy()
    dist_matrix = np.zeros(shape=(len(df), len(df)))
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            dist_matrix[i, j] = round(sqrt((coordinates[i, 0] - coordinates[j, 0])**2 + (coordinates[i, 1] - coordinates[j, 1])**2))
    return dist_matrix

In [4]:
def calculate_nearest_vertices(dist_matrix: np.ndarray, costs: np.ndarray, num_candidates: int = 10) -> dict:
    num_vertices = dist_matrix.shape[0]
    nearest_vertices = []

    for i in range(num_vertices):
        cost_distance_sum = dist_matrix[i, :] + costs
        sorted_vertices = np.argsort(cost_distance_sum)
        nearest_vertices.append([vertex for vertex in sorted_vertices if vertex != i][:num_candidates])

    return nearest_vertices

In [5]:
distances_matrices = {
    "A": calculate_distance_matrix(instances["A"]),
    "B": calculate_distance_matrix(instances["B"]),
    "C": calculate_distance_matrix(instances["C"]),
    "D": calculate_distance_matrix(instances["D"])
}

costs = {
    "A": instances["A"]["cost"].to_numpy(),
    "B": instances["B"]["cost"].to_numpy(),
    "C": instances["C"]["cost"].to_numpy(),
    "D": instances["D"]["cost"].to_numpy()
}

nearest_vertices = {
    "A": calculate_nearest_vertices(distances_matrices["A"], costs["A"]),
    "B": calculate_nearest_vertices(distances_matrices["B"], costs["B"]),
    "C": calculate_nearest_vertices(distances_matrices["C"], costs["C"]),
    "D": calculate_nearest_vertices(distances_matrices["D"], costs["D"])
}

In [6]:
def visualize_selected_route(
    selected_nodes_indices: ArrayLike, 
    dataframe: pd.DataFrame,
    title: str) -> None:
    """
    Visualize the selected route returned by the algorithm, including the cost of each node represented by a colormap.

    Parameters:
    selected_nodes_indices (list): Indices of the selected nodes in the route.
    dataframe (DataFrame): DataFrame containing 'x', 'y', and 'cost' columns for each node.
    """
    x = dataframe["x"].to_numpy()
    y = dataframe["y"].to_numpy()
    costs = dataframe["cost"].to_numpy()

    cmap = plt.cm.get_cmap('viridis')
    norm = plt.Normalize(vmin=min(costs), vmax=max(costs))

    plt.figure(figsize=(15, 10))
    scatter = plt.scatter(x, y, c=costs, cmap=cmap, norm=norm, s=100)
    plt.colorbar(scatter, label='Node Cost')

    for i, node in enumerate(selected_nodes_indices):
        start_node = selected_nodes_indices[i]
        end_node = selected_nodes_indices[(i + 1) % len(selected_nodes_indices)]
        plt.plot([x[start_node], x[end_node]], [y[start_node], y[end_node]], 'k-', lw=1)

    plt.title(title, fontsize=18)
    plt.xlabel('X Coordinate', fontsize=14)
    plt.ylabel('Y Coordinate', fontsize=14)
    plt.grid(True)
    plt.show()

In [7]:
def objective_function(solution: list[int], dist_matrix: list[list[int]], costs: list[int]) -> int:
    total_score = 0
    n = len(solution)
    for x in range(n):
        total_score += dist_matrix[solution[x - 1]][solution[x]]
        total_score += costs[solution[x]]
    return total_score

In [8]:
# A function that generates a random solution
def generate_random_solution(n: int) -> list[int]:
    """
    Generate a random solution for a given number of nodes.

    :param n: The number of nodes.
    :return: A list of nodes representing the solution.
    """
    return random.sample(range(0, n * 2), n)

In [88]:
def two_edges_exchange(current_solution: list[int], 
                       nearest_vertices: list[list[int]],
                       score,
                       distance_matrix,
                       cost) -> list[list[int]]:
    """
    Generate new solutions by exchanging two edges in the current solution.

    :param current_solution: List of nodes in the current solution.
    :param current_score: The score of the current solution.
    :param distance_matrix: 2D list representing the distances between nodes.
    :return: A list of tuples where each tuple contains a new solution and its score.
    """
    n = len(current_solution)
    new_solutions = []

    for i in range(n):
        node1 = current_solution[i]
        node1_index = i
        node2_candidates = nearest_vertices[node1]
        
        for node2 in node2_candidates:
            if node2 in current_solution:
                node2_index = current_solution.index(node2)
                print(node2, node1)

                if node2_index < node1_index:
                    node1_index, node2_index = node2_index, node1_index
                
                if (node2_index - node1_index) % n > 1:
                        
                    new_solution1 = (current_solution[:node1_index] 
                                    + current_solution[node1_index :node2_index][::-1] 
                                    + current_solution[node2_index  :])
                    
                    new_solution2 = (current_solution[:node1_index + 1] 
                                    + current_solution[node2_index:] 
                                    + current_solution[node1_index + 1:node2_index][::-1])
                    
                    new_solutions.append(new_solution1)
                    new_solutions.append(new_solution2)
    
    return new_solutions

In [90]:
sol = [2,7,4,6,1,5,3,8,0]
nearest_vertics = [
    [7],[],[],[],[],[],[],[],[]
]
score=0 
dis=distances_matrices["A"]
cost=costs["A"]
two_edges_exchange(sol, nearest_vertics, score, dis, cost)

7 0


[[2, 8, 3, 5, 1, 6, 4, 7, 0], [2, 7, 0, 8, 3, 5, 1, 6, 4]]

In [78]:
def inter_route_exchange_simple(selected, unselected, score, distance_matrix, costs,  nearest_vertices):
    new_solutions = []

    # Assume node indices are 0-based for distance_matrix
    for selected_node in selected:
        for node in nearest_vertices[selected_node]:
            if node in unselected:
                new_solution1 = selected.copy()
                new_solution2 = selected.copy()
                replaced_node_index1= selected.index(selected_node)-1
                replaced_node_index2= selected.index(selected_node)+1

            
                # Assuming 'selected' and 'unselected' contain 0-based indices already
                new_solution1[replaced_node_index1] = node
                new_solution2[replaced_node_index2] = node

                prev_node_index1 = (replaced_node_index1 - 1) % len(selected)
                next_node_index1 = (replaced_node_index1 + 1) % len(selected)

                prev_node_index2 = (replaced_node_index2 - 1) % len(selected)
                next_node_index2 = (replaced_node_index2 + 1) % len(selected)

                # Calculate score_delta considering 0-based indices for distance_matrix
                score_delta1 = (
                    -distance_matrix[selected[prev_node_index1]][selected_node]
                    -distance_matrix[selected_node][selected[next_node_index1]]
                    +distance_matrix[selected[prev_node_index1]][node]
                    +distance_matrix[node][selected[next_node_index1]]
                    -costs[selected_node]
                    +costs[node]
                )
                score_delta2 = (
                    -distance_matrix[selected[prev_node_index2]][selected_node]
                    -distance_matrix[selected_node][selected[next_node_index2]]
                    +distance_matrix[selected[prev_node_index2]][node]
                    +distance_matrix[node][selected[next_node_index2]]
                    -costs[selected_node]
                    +costs[node]
                )
            new_score1 = score + score_delta1

            new_solutions.append((new_solution1, new_score1))
            new_score2= score + score_delta2

            new_solutions.append((new_solution2, new_score2))

    return new_solutions


In [79]:
sol = [ x for x in range (10)]
uns = [x for x in range (20,25)]
score=0 
dis=distances_matrices["A"]
cost=costs["A"]

nearest_vertics = [
    [23],[],[],[],[],[],[],[],[],[]
]
inter_route_exchange_simple(sol, uns, score, dis, cost,  nearest_vertics)

[([0, 1, 2, 3, 4, 5, 6, 7, 8, 23], 3646.0),
 ([0, 23, 2, 3, 4, 5, 6, 7, 8, 9], 3768.0)]

# Repository for Local Search solution finder #

In [7]:
from abc import ABC, abstractmethod
from typing import Union, Optional


class LocalSearch(ABC):
    """
    Abstract class for local search algorithms.
    """
    def __init__(self, 
                 initial_solution: list[int], 
                 distance_matrix: list[list[int]], 
                 costs: list[int]) -> None:
        """
        :param initial_solution: The initial solution.
        :param initial_score: The score of the initial solution.
        :param distance_matrix: 2D list representing the distances between nodes.
        """
        self.initial_solution = initial_solution
        self.initial_score = objective_function(initial_solution, distance_matrix, costs)
        self.distance_matrix = distance_matrix
        self.costs = costs
        
        self.all_nodes = list(range(100*2))
        self.unselected_nodes = [node for node in self.all_nodes if node not in self.initial_solution]
        
        self.current_solution: list[int] = initial_solution
        self.current_score = self.initial_score
        
        self.moves = {
            "intra-two-edges-exchange": self.two_edges_exchange,
            "intra-two-nodes-exchange": self.two_nodes_exchange,
            "inter-route-exchange": self.inter_route_exchange
        }
        
        self.moves_utils = {
            "intra-two-edges-exchange": self.get_utils_intra_two_edges_exchange,
            "intra-two-nodes-exchange": self.get_utils_intra_two_nodes_exchange,
            "inter-route-exchange": self.get_utils_inter_route_exchange
        }
        
    
    @abstractmethod
    def candidates_moves
        
        
    @abstractmethod
    def get_utils_intra_two_edges_exchange(self):
        raise NotImplementedError
    
    @abstractmethod
    def get_utils_intra_two_nodes_exchange(self):
        raise NotImplementedError
    
    @abstractmethod
    def get_utils_inter_route_exchange(self):
        raise NotImplementedError

    # two method for intra moves
    @abstractmethod
    def two_edges_exchange(self, start_index: int = 0, direction: str = "right") -> Union[tuple[list[int], int], None]:
        """This method should return a new solution and its score if a better solution is found, otherwise None.
        For a greedy algorithm, the first better solution found should be returned. For a steepest local search,
        the best solution among all possible solutions should be returned, if there is no better solution, just
        return None. For a steepest local search, both neighborhood moves will return one solution, then we will
        take the better one among these two solutions.

        Parameters
        ----------
        current_solution : list[int]
        current_distance : int
        distance_matrix : list[list[int]]

        Returns
        -------
        tuple[list[int], int] | None
            A single better solution founded by the algorithm, or None if no better solution is found.
        """
        raise NotImplementedError
    
    @abstractmethod
    def two_nodes_exchange(self, start_index=0, direction='right') -> Union[tuple[list[int], int] , None]:
        raise NotImplementedError
    
    # one method for inter move
    @abstractmethod
    def inter_route_exchange(self, start_index=0, direction="right") -> Union[tuple[list[int], int] , None]:
        raise NotImplementedError
    
    @abstractmethod
    def run(self, *args, **kwargs) -> None:
        """
        Run the algorithm.
        """
        pass

In [8]:
class CandidateLocalSearch(LocalSearch):
    def __init__(self, 
                 initial_solution: list[int], 
                 distance_matrix: list[list[int]], 
                 costs: list[int]) -> None:
        super().__init__(initial_solution, distance_matrix, costs)
        self.nearest_vertices = self.candidate_nodes

    def candidate_nodes(self, 
                        num_candidates):
        num_sol = len(self.distance_matrix[0])
        self.candidates = [[] for x in range(num_sol)]
        for x in range( num_sol ):
           distance_and_cost=distances_matrices["A"][x] + costs["A"]
           temp = sorted(range(num_sol), key=lambda k: distance_and_cost[k])[:num_candidates+1]
           if x in temp:
            temp.remove(x)
           else:
            temp.remove(temp[-1])
           self.candidates[x] = temp 


        

In [107]:
num_candidates=10
num_sol = len(distances_matrices["A"])

candidates = [[] for x in range(num_sol)]

for x in range( num_sol ):
    distance_and_cost=distances_matrices["A"][x] + costs["A"]
    temp = sorted(range(num_sol), key=lambda k: distance_and_cost[k])[:num_candidates+1]
    
    if x in temp:
        temp.remove(x)
    else:
        temp.remove(temp[-1])
    candidates[x] = temp 

for x in candidates[0]:
    distance_and_cost=distances_matrices["A"][0] + costs["A"]
    print(distance_and_cost[x])


447.0
547.0
562.0
695.0
760.0
895.0
906.0
960.0
972.0
988.0


In [97]:
costs["A"][115]

1415